# Reforma

## Preâmbulo

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import time

# Track execution time                                                          
start_time = time.time()

### Parâmetros

In [2]:
# Simulation Parameters
#TB_REFORMA = 'FATO_REFORMA_SAMPLE'
TB_REFORMA = 'FATO_REFORMA'
ANO_INICIO = 1995
ANO_FIM = 2016
DADOS_FAZENDA = '../dataset/dados_fazenda.xlsx'

# Conection parameters
HOST='tama'
PORT='5432'
DBNAME='prevdb'
USER='prevdb_user'
PASS='pr3v'

### Biblioteca

In [3]:
def ds_query(sql_query):
    """
        Query Dataset

    Parâmetros
    ----------
        sql : string
            SQL query to be performed against the dataset
        
    Retorno
    -------
        Pandas Dataframe
    """
    # Connect to an existing database
    try:
        conn = psycopg2.connect("host='{}' port={} dbname='{}'user={} password={}"
                .format(HOST, PORT, DBNAME, USER, PASS))
        df = sqlio.read_sql_query(sql, conn)
        # Close communication with the database                                     
        conn.close()
        return df
    except:
        print("Unable to connect to the database")
        return

## Simulação

### FATO_REFORMA

In [4]:
sql = """
SELECT *
FROM {table_name} 
LIMIT 10
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
fato_pessoa = ds_query(sql)
fato_pessoa

,index,ano_nasc,dt_nasc,dt_obito,sexo,clientela,ano_inicio_contrib,ano_dib,idade_dib,tempo_contrib,especie,pec6_ano_dib,pec6_idade_dib,pec6_gap,pec6_prob,pec6_percent
0,832,1958,19580513,20160617,3,1,2006,2013,55,7,32,2013.0,55.0,0,1.000000,0.60
1,833,1966,19660919,0,1,1,2011,2015,48,4,32,2015.0,48.0,0,1.000000,0.60
2,834,1964,19640618,0,1,1,1986,2015,50,29,46,2024.0,60.0,10,0.910483,0.98
3,835,1956,19561117,0,3,2,2012,2012,55,0,41,2032.0,76.0,21,0.755011,0.62
4,836,1975,19750206,0,3,1,2007,2015,40,8,32,2015.0,40.0,0,1.000000,0.60
5,837,1952,19520902,0,1,1,1979,2015,62,36,42,2017.0,65.0,3,0.949612,0.98
6,838,1952,19520610,0,1,2,2012,2012,60,0,41,2032.0,80.0,20,0.535958,0.60
7,839,1949,19490329,0,3,1,2005,2013,64,8,32,2013.0,64.0,0,1.000000,0.60
8,840,1956,19561212,0,1,1,2008,2013,56,5,32,2013.0,56.0,0,1.000000,0.60
9,841,1954,19540112,0,1,1,1980,2015,61,35,42,2019.0,65.0,4,0.935076,0.98


### Aposentados RGPS em ANO_FIM

In [5]:
sql = """
SELECT 
	ESPECIE
    ,CLIENTELA
	,SEXO
    ,IDADE_DIB
	,PEC6_IDADE_DIB
	--,PEC6_TEMPO_CONTRIB
	,PEC6_GAP
	,PEC6_ANO_DIB
	,PEC6_PROB
    ,PEC6_PERCENT
FROM {table_name} 
WHERE ANO_DIB = {ano}
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
fato_pessoa = ds_query(sql)
print(fato_pessoa.columns)
print(fato_pessoa.shape)

Index(['especie', 'clientela', 'sexo', 'idade_dib', 'pec6_idade_dib',
       'pec6_gap', 'pec6_ano_dib', 'pec6_prob', 'pec6_percent'],
      dtype='object')
(889823, 9)


In [6]:
print('ANO FIM = {}'.format(ANO_FIM))
df = fato_pessoa[['especie','clientela', 'sexo', 'idade_dib', 'pec6_idade_dib','pec6_prob','pec6_gap','pec6_percent']]
print(df['especie'].unique())

ANO FIM = 2016
[41 42 57 32 46 92]


In [7]:
df_rgps = df.pivot_table(index='idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_prob', aggfunc='count').round()
df_rgps.fillna(value=0, inplace=True, downcast='infer')
df_rgps

especie      32                  41                42        ...      46      \
clientela     1         2         1        2        1        ...  2    1       
sexo          1    3    1    3    1    3   1   3    1     3  ...  3    1   3   
idade_dib                                                    ...               
15            1    0    0    0    0    0   0   0    0     0  ...  0    0   0   
17            1    0    0    1    0    0   0   0    0     0  ...  0    0   0   
18            8    1    1    0    0    0   0   0    0     0  ...  0    0   0   
19           21    2    2    0    0    0   0   0    0     0  ...  0    0   0   
20           38    6    7    2    0    0   0   0    0     0  ...  0    0   0   
21           54    7   11    2    0    0   0   0    0     0  ...  0    0   0   
22           85   20   16    5    0    0   0   0    0     0  ...  0    0   0   
23          129   31   12    4    0    0   0   0    0     0  ...  0    0   0   
24          143   54   15    6    0    0   0   0    0     0  ...  0    0   0   
25          166   72   19   16    0    0   0   0    0     0  ...  0    0   0   
26          211   59   31   20    0    0   0   0    0     0  ...  0    0   0   
27          263   98   27   16    0    0   0   0    0     0  ...  0    0   0   
28          263  119   23   17    0    0   0   0    0     0  ...  0    0   0   
29          312  137   30   22    0    0   0   0    0     0  ...  0    0   0   
30          353  181   40   35    0    0   0   0    0     0  ...  0    0   0   
31          417  221   39   32    0    0   0   0    0     0  ...  0    0   0   
32          436  208   50   43    0    0   0   0    0     0  ...  0    0   0   
33          482  286   60   38    0    0   0   0    0     0  ...  0    0   0   
34          513  306   57   54    0    0   0   0    0     0  ...  0    0   0   
35          551  362   50   45    0    0   0   0    0     0  ...  0    4   0   
36          601  394   63   63    0    0   0   0    0     0  ...  0    7   0   
37          612  432   62   62    0    0   0   0    0     0  ...  0   12   0   
38          700  447   81   66    0    0   0   0    1     0  ...  0    6   0   
39          713  521   69   69    0    0   0   0    0     3  ...  0   18   0   
40          810  536   79   85    0    0   0   0   13    15  ...  1   31   3   
41          772  561  103   93    0    0   0   0   38    48  ...  0   48   5   
42          884  651  115   89    0    0   0   0   54   273  ...  1   72  12   
43         1042  694  100  108    0    0   0   0  143   441  ...  0  135  25   
44         1111  704  107  110    0    0   0   0  277   743  ...  1  187  27   
45         1271  806  134  112    0    0   0   0  480  1264  ...  3  269  45   
...         ...  ...  ...  ...  ...  ...  ..  ..  ...   ...  ... ..  ...  ..   
75           34   93    2    5  217  297  53  64   13     4  ...  0    0   0   
76           25   59    0    3  178  279  51  64   19     1  ...  0    0   0   
77           25   51    1    3   99  184  36  52    7     4  ...  0    1   0   
78           20   33    1    2  107  159  29  35    7     1  ...  0    0   0   
79            8   18    1    2   72  132  31  38    3     4  ...  0    0   0   
80            7   33    1    2   86  107  14  31    5     2  ...  0    0   0   
81           11   12    0    3   72   85  28  18    2     0  ...  0    0   0   
82            5   10    0    0   53   56  11  18    2     0  ...  0    0   0   
83            1    6    1    0   40   40   9  22    2     0  ...  0    0   0   
84            2    5    0    0   26   37  11  18    0     1  ...  0    0   0   
85            4    4    0    0   29   28  15  14    3     0  ...  0    0   0   
86            1    4    0    0   16   25   4  14    2     0  ...  0    0   0   
87            1    3    0    0   11   17   6  14    0     1  ...  0    0   0   
88            1    2    1    1    6   11  10   6    0     0  ...  0    0   0   
89            0    2    0    0    4    7   2  12    0     0  ...  0    0   0   


### Aposentados PEC 6/2019 em ANO_FIM

In [8]:
sql = """
SELECT 
	ESPECIE
    ,CLIENTELA
	,SEXO
    ,IDADE_DIB
	,PEC6_IDADE_DIB
	--,PEC6_TEMPO_CONTRIB
	,PEC6_GAP
	,PEC6_ANO_DIB
	,PEC6_PROB
    ,PEC6_PERCENT
FROM {table_name} 
WHERE PEC6_ANO_DIB = {ano}
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
fato_pessoa = ds_query(sql)
print(fato_pessoa.columns)
print(fato_pessoa.shape)

Index(['especie', 'clientela', 'sexo', 'idade_dib', 'pec6_idade_dib',
       'pec6_gap', 'pec6_ano_dib', 'pec6_prob', 'pec6_percent'],
      dtype='object')
(921862, 9)


In [9]:
print('ANO FIM = {}'.format(ANO_FIM))
df = fato_pessoa[['especie','clientela', 'sexo', 'idade_dib', 'pec6_idade_dib','pec6_prob','pec6_gap','pec6_percent']]
print(df['especie'].unique())

ANO FIM = 2016
[41 42 57 32 46 92]


In [10]:
df_qtd = df.pivot_table(index='pec6_idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_prob', aggfunc='sum').round()
df_qtd.fillna(value=0, inplace=True, downcast='infer')
#df = df.groupby(['pec6_idade_dib','sexo']).sum()
#df['qtd'] = df['pec6_prob'].apply(lambda x: round(x))
#df_qtd.to_csv('../sandbox/2016_pec_qtd_sample.csv')
df_qtd

especie           32                  41                42     ...    46     \
clientela          1         2         1         2       1     ...  2  1      
sexo               1    3    1    3    1    3    1    3  1  3  ...  3  1  3   
pec6_idade_dib                                                 ...            
15.0               1    0    0    0    0    0    0    0  0  0  ...  0  0  0   
17.0               1    0    0    1    0    0    0    0  0  0  ...  0  0  0   
18.0               8    1    1    0    0    0    0    0  0  0  ...  0  0  0   
19.0              21    2    2    0    0    0    0    0  0  0  ...  0  0  0   
20.0              38    6    7    2    0    0    0    0  0  0  ...  0  0  0   
21.0              54    7   11    2    0    0    0    0  0  0  ...  0  0  0   
22.0              85   20   16    5    0    0    0    0  0  0  ...  0  0  0   
23.0             129   31   12    4    0    0    0    0  0  0  ...  0  0  0   
24.0             143   54   15    6    0    0    0    0  0  0  ...  0  0  0   
25.0             166   72   19   16    0    0    0    0  0  0  ...  0  0  0   
26.0             211   59   31   20    0    0    0    0  0  0  ...  0  0  0   
27.0             263   98   27   16    0    0    0    0  0  0  ...  0  0  0   
28.0             263  119   23   17    0    0    0    0  0  0  ...  0  0  0   
29.0             312  137   30   22    0    0    0    0  0  0  ...  0  0  0   
30.0             353  181   40   35    0    0    0    0  0  0  ...  0  0  0   
31.0             417  221   39   32    0    0    0    0  0  0  ...  0  0  0   
32.0             436  208   50   43    0    0    0    0  0  0  ...  0  0  0   
33.0             482  286   60   38    0    0    0    0  0  0  ...  0  0  0   
34.0             513  306   57   54    0    0    0    0  0  0  ...  0  0  0   
35.0             551  362   50   45    0    0    0    0  0  0  ...  0  0  0   
36.0             601  394   63   63    0    0    0    0  0  0  ...  0  0  0   
37.0             612  432   62   62    0    0    0    0  0  0  ...  0  0  0   
38.0             700  447   81   66    0    0    0    0  0  0  ...  0  0  0   
39.0             713  521   69   69    0    0    0    0  0  0  ...  0  0  0   
40.0             810  536   79   85    0    0    0    0  0  0  ...  0  0  0   
41.0             772  561  103   93    0    0    0    0  0  0  ...  0  0  0   
42.0             884  651  115   89    0    0    0    0  0  0  ...  0  0  0   
43.0            1042  694  100  108    0    0    0    0  0  0  ...  0  0  0   
44.0            1111  704  107  110    0    0    0    0  0  0  ...  0  0  0   
45.0            1271  806  134  112    0    0    0    0  0  0  ...  0  0  0   
...              ...  ...  ...  ...  ...  ...  ...  ... .. ..  ... .. .. ..   
81.0              11   12    0    3  225  368  502  258  4  0  ...  0  0  0   
82.0               5   10    0    0  186  211  265  196  2  0  ...  0  0  0   
83.0               1    6    1    0  114  127  124  165  3  0  ...  0  0  0   
84.0               2    5    0    0  100  123   89  156  1  0  ...  0  0  0   
85.0               4    4    0    0  273   88   84  137  1  1  ...  0  0  0   
86.0               1    4    0    0  131   69   84  123  4  0  ...  0  0  0   
87.0               1    3    0    0   51   41   48   92  0  0  ...  0  0  0   
88.0               1    2    1    1   32   42   31   75  0  1  ...  0  0  0   
89.0               0    2    0    0   24   29   19   64  0  0  ...  0  0  0   
90.0               0    2    0    0   17   26    6   55  0  0  ...  0  0  0   
91.0               1    0    0    0   13   28    7   39  0  0  ...  0  0  0   
92.0               1    0    0    0   10   18    5   34  0  0  ...  0  0  0   
93.0               0    0    0    0    6   12    2   32  0  0  ...  0  0  0   
94.0               0    0    0    0    5   11    2   24  0  0  ...  0  0  0   
95.0               0    0    0    1    7    8    1   24  0  0  ...  0  0  0   
96.0               0    0    0    0    5    4    2

In [11]:
df_qtd.columns

MultiIndex(levels=[[32, 41, 42, 46, 57, 92], [1, 2], [1, 3]],
           codes=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 5], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]],
           names=['especie', 'clientela', 'sexo'])

### GAP Médio PEC 6/2019 em ANO_FIM

In [12]:
df_gap = df.pivot_table(index='idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_gap', aggfunc='mean')#.round()
df_gap.fillna(value=0, inplace=True, downcast='infer')
#df_qtd.to_csv('../sandbox/2016_pec_avggap_sample.csv')
df_gap

especie   32                 41                                        42  \
clientela  1     2            1                   2                     1   
sexo       1  3  1  3         1         3         1          3          1   
idade_dib                                                                   
15         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
17         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
18         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
19         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
20         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
21         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
22         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
23         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
24         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
25         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
26         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
27         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
28         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
29         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
30         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
31         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
32         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
33         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
34         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
35         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
36         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
37         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
38         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
39         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
40         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
41         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
42         0  0  0  0  0.000000  0.000000  0.000000   0.000000   0.000000   
43         0  0  0  0  0.000000  0.000000  0.000000   0.000000  22.000000   
44         0  0  0  0  0.000000  0.000000  0.000000   0.000000  21.000000   
45         0  0  0  0  0.000000  0.000000  0.000000   0.000000  20.000000   
...       .. .. .. ..       ...       ...       ...        ...        ...   
69         0  0  0  0  4.669765  5.903226  8.307263  12.127413   0.582609   
70         0  0  0  0  4.834513  6.021589  7.287879   9.724444   0.564516   
71         0  0  0  0  5.389896  5.957311  5.032967  10.774510   0.566667   
72         0  0  0  0  5.603478  6.119617  6.512500  10.820690   0.640000   
73         0  0  0  0  5.571739  6.661765  6.301587  10.668966   0.571429   
74         0  0  0  0  6.212202  6.263314  4.711111   9.571429   0.600000   
75         0  0  0  0  6.112628  6.329710  4.555556   8.137931   0.692308   
76         0  0  0  0  5.760563  6.852041  3.882353   8.153846   0.842105   
77         0  0  0  0  6.506250  6.656489  3.468750   8.458824   0.857143   
78         0  0  0  0  5.877049  6.772277  3.411765   9.389831   0.857143   
79         0  0  0  0  5.758621  7.756757  3.650000   8.659574   0.333333   
80         0  0  0  0  5.271429  8.166667  2.000000   7.934783   0.800000   
81         0  0  0  0  4.631579  8.055556  2.916667   9.166667   1.000000   
82         0  0  0  0  4.977273  6.483871  2.375000   7.937500   1.000000   
83         0  0  0  0  5.227273  7.157895  3.166667   4.600000   0.500000   
84         0  0  0  0  6.111111  6.647059  3.333333   7.666667   0.000000   
85         0  0  0  0  6.125000  5.105263  0.800000   8.047619   0.666667

### Probabilidade de Sobrevivência Média PEC 6/2019 em ANO_FIM

In [13]:
df_prob = df.pivot_table(index='idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_prob', aggfunc='mean')#.round()
df_prob.fillna(value=0, inplace=True, downcast='infer')
#df_prob.to_csv('../sandbox/2016_pec_avgprob_sample.csv')
df_prob

especie   32                 41                                      42  \
clientela  1     2            1                   2                   1   
sexo       1  3  1  3         1         3         1         3         1   
idade_dib                                                                 
15         1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
17         1  0  0  1  0.000000  0.000000  0.000000  0.000000  0.000000   
18         1  1  1  0  0.000000  0.000000  0.000000  0.000000  0.000000   
19         1  1  1  0  0.000000  0.000000  0.000000  0.000000  0.000000   
20         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
21         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
22         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
23         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
24         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
25         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
26         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
27         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
28         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
29         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
30         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
31         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
32         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
33         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
34         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
35         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
36         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
37         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
38         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
39         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
40         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
41         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
42         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
43         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.786208   
44         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.787199   
45         1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.789051   
...       .. .. .. ..       ...       ...       ...       ...       ...   
69         1  1  1  1  0.833749  0.854767  0.688317  0.619879  0.982984   
70         1  1  1  1  0.815333  0.838425  0.725499  0.701114  0.980044   
71         1  1  1  1  0.780492  0.828662  0.802758  0.662925  0.979970   
72         1  1  1  1  0.757711  0.808730  0.736296  0.652675  0.977380   
73         1  1  1  1  0.741069  0.771723  0.743295  0.660055  0.979802   
74         1  1  1  1  0.690635  0.765472  0.781186  0.666805  0.973448   
75         1  1  1  1  0.673764  0.743366  0.799231  0.720839  0.962894   
76         1  1  0  1  0.682622  0.700885  0.820570  0.734650  0.954870   
77         1  1  1  1  0.625726  0.692679  0.828228  0.717711  0.954092   
78         1  1  1  1  0.646221  0.671759  0.798343  0.694467  0.954087   
79         1  1  1  1  0.620353  0.602674  0.795043  0.726619  0.978047   
80         1  1  1  1  0.648172  0.599878  0.854175  0.727998  0.936844   
81         1  1  0  1  0.686547  0.589426  0.826452  0.723866  0.921121   
82         1  1  0  0  0.667159  0.674174  0.811233  0.785201  0.921224   
83         1  1  1  0  0.715136  0.702799  0.743160  0.813264  0.960650   
84         1  1  0  0  0.703272  0.744536  0.748988  0.865588  0.000000   
85         1  1  0  0  0.743422  0.821702  0.907535  0.995531  0.922946   
86         1  1  0  0  0.870565  0.986101  0.982092  0.977095  1.000000   
87         1  1  0  0  1.

### Percentual Médio PEC 6/2019

In [14]:
df_ppc = df.pivot_table(index='pec6_idade_dib', columns=['especie','clientela','sexo'], 
                        values='pec6_percent', aggfunc='mean')#.round()
df_ppc.fillna(value=0, inplace=True, downcast='infer')
#df_prob.to_csv('../sandbox/2016_pec_avgprob_sample.csv')
df_ppc

especie               32                                      41            \
clientela              1                   2                   1             
sexo                   1         3         1         3         1         3   
pec6_idade_dib                                                               
15.0            0.600000  0.000000  0.000000  0.000000  0.000000  0.000000   
17.0            0.600000  0.000000  0.000000  0.600000  0.000000  0.000000   
18.0            0.600000  0.600000  0.600000  0.000000  0.000000  0.000000   
19.0            0.600000  0.600000  0.600000  0.000000  0.000000  0.000000   
20.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
21.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
22.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
23.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
24.0            0.600000  0.600000  0.602667  0.600000  0.000000  0.000000   
25.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
26.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
27.0            0.600000  0.600000  0.600000  0.605000  0.000000  0.000000   
28.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
29.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
30.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
31.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
32.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
33.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
34.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
35.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
36.0            0.600000  0.600000  0.600000  0.600000  0.000000  0.000000   
37.0            0.600000  0.600046  0.600000  0.602903  0.000000  0.000000   
38.0            0.600029  0.600000  0.600494  0.600000  0.000000  0.000000   
39.0            0.600168  0.600000  0.600000  0.600000  0.000000  0.000000   
40.0            0.600272  0.600075  0.602278  0.600471  0.000000  0.000000   
41.0            0.600233  0.600428  0.601553  0.601720  0.000000  0.000000   
42.0            0.601357  0.600922  0.600174  0.600899  0.000000  0.000000   
43.0            0.602035  0.600893  0.602600  0.602778  0.000000  0.000000   
44.0            0.602970  0.601903  0.603178  0.603091  0.000000  0.000000   
45.0            0.605020  0.602084  0.602239  0.606250  0.000000  0.000000   
...                  ...       ...       ...       ...       ...       ...   
81.0            0.600000  0.600000  0.000000  0.600000  0.618519  0.612750   
82.0            0.672000  0.600000  0.000000  0.000000  0.617967  0.612086   
83.0            0.600000  0.600000  0.600000  0.000000  0.617200  0.610746   
84.0            0.600000  0.600000  0.000000  0.000000  0.612581  0.613663   
85.0            0.600000  0.600000  0.000000  0.000000  0.602550  0.610588   
86.0            0.600000  0.600000  0.000000  0.000000  0.615756  0.612632   
87.0            0.600000  0.600000  0.000000  0.000000  0.616143  0.610976   
88.0            0.600000  0.600000  0.600000  0.600000  0.617582  0.611087   
89.0            0.000000  0.600000  0.000000  0.000000  0.610000  0.613333   
90.0            0.000000  0.600000  0.000000  0.000000  0.610213  0.612800   
91.0            0.600000  0.000000  0.000000  0.000000  0.611613  0.609804   
92.0            0.600000  0.000000  0.000000  0.000000  0.610000  0.609655   
93.0            0.000000  0.000000  0.000000  0.000000  0.658000  0.618750   
94.0            0.000000  0.000000  0.000000  0.000000  0.603333  0.620000   
95.0            0.000000  0.000000  0.000000  0.600000  0.612500  0.608889   
96.0            0.000000  0.000000  0.000000  0.000000  0.616667  0.605000   
97.0            0.0000

## Considerações

* **Probabilidade de a pessoa não querer se aposentar na idade mínima**
 * Utilizar probabilidades da base?
 * Como mensurar a "vontade de trabalhar mais para aumentar o valor do benefício"?
* **Aposentadoria por idade de homens**
 * Não sofre alterações em termos de quantidade na PEC 6/2019?
* **Tempo de Contribuição**
 * É válido condisiderar que o ano de inicio de contribuição como:  `ano_inicio_contrib = ano_dib - tempo_contrib`
 * Essa abordagem desconsidera períodos de desemprego/i

## Cleanup

In [15]:
# Print out elapsed time                                                        
elapsed_time = (time.time() - start_time) / 60                                  
print("\nExecution time: {0:0.4f} minutes.".format(elapsed_time))


Execution time: 0.1579 minutes.
